In [176]:
try:
    import re
    import typing
    from urllib import (request, error)
    import sys
    from bs4 import BeautifulSoup
except ModuleNotFoundError as err:
    raise ModuleNotFoundError(err.__dict__)
    sys.exit(1)

In [178]:
def fetch_llvm_releases_github_page() -> str:
    page: str = ""
    LLVM_RELEASE_PAGE = r"https://github.com/llvm/llvm-project/releases"
    req = request.Request(url = LLVM_RELEASE_PAGE, method = "GET", headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/110.0"
    })
    try:
        with request.urlopen(req) as response:
            page = str(response.read())
    except error.HTTPError as err:
        raise error.HTTPError(err.__dict__) 
    return page

In [179]:
fetch_llvm_releases_github_page();

In [180]:
def extract_llvm_release_versions_and_links(html_document: str) -> typing.Dict[str, str]:
    
    """
    A function that takes GitHub LLVM releases HTML page as a string and returns
    a list of links to actually access the download URIs.
    Requires re and BeautifulSoup.
    """
    
    if not isinstance(html_document, str):
        raise TypeError("Incompatible types. Argument must be of string <class 'str'> type.")
        
    links: typing.Dict[str, str] = dict()
    
    soup = BeautifulSoup(html_document, "html.parser")
    
    for section in soup.find_all("section"):
        version = section.find("h2", attrs = {"class": "sr-only"}).text
        include_fragment = section.find("include-fragment", attrs = {"loading": "lazy"})
        
        if include_fragment:
            links[version] = include_fragment.get("src")
            
        else:
            lazy_load = section.find("include-fragment", attrs = {"class": "js-truncated-assets-fragment"})
            links[version] = lazy_load.get("data-deferred-src")
    return links

In [181]:
extract_llvm_release_versions_and_links(page)

{'LLVM 16.0.0-rc3': 'https://github.com/llvm/llvm-project/releases/expanded_assets/llvmorg-16.0.0-rc3',
 'LLVM 16.0.0-rc2': 'https://github.com/llvm/llvm-project/releases/expanded_assets/llvmorg-16.0.0-rc2',
 'LLVM 16.0.0-rc1': 'https://github.com/llvm/llvm-project/releases/expanded_assets/llvmorg-16.0.0-rc1',
 'LLVM 15.0.7': 'https://github.com/llvm/llvm-project/releases/expanded_assets/llvmorg-15.0.7',
 'LLVM 15.0.6': 'https://github.com/llvm/llvm-project/releases/expanded_assets/llvmorg-15.0.6',
 'LLVM 15.0.5': 'https://github.com/llvm/llvm-project/releases/expanded_assets/llvmorg-15.0.5',
 'LLVM 15.0.4': 'https://github.com/llvm/llvm-project/releases/expanded_assets/llvmorg-15.0.4',
 'LLVM 15.0.3': 'https://github.com/llvm/llvm-project/releases/expanded_assets/llvmorg-15.0.3',
 'LLVM 15.0.2': 'https://github.com/llvm/llvm-project/releases/expanded_assets/llvmorg-15.0.2',
 'LLVM 15.0.1': 'https://github.com/llvm/llvm-project/releases/expanded_assets/llvmorg-15.0.1'}

In [186]:
def get_llvm_win64_download_uris(urls: list[str]) -> list[str]:
    
    """
    
    """
    
    download_uris: list[str] = list()

    BASE_URL = "https://github.com"
    win64 = re.compile(r"/llvm/llvm-project/releases/download/[\d\w\-\.\/]*win64.exe")
    
    for url in urls:
        req_custom = request.Request(url = url, method = "GET", headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/110.0"
        })
    
        with request.urlopen(req_custom) as response:
            try:
                resp_body = str(response.read().decode("utf8"))
                download_link = re.findall(win64, resp_body)
                if download_link:
                    download_uris.append(BASE_URL + download_link[0])
                else:
                    download_uris.append(None)
                
            except error.HTTPError as err:
                print(err)
    return download_uris

In [187]:
get_llvm_win64_download_uris(extract_llvm_release_links(page))

['https://github.com/llvm/llvm-project/releases/download/llvmorg-16.0.0-rc3/LLVM-16.0.0-rc3-win64.exe',
 'https://github.com/llvm/llvm-project/releases/download/llvmorg-16.0.0-rc2/LLVM-16.0.0-rc2-win64.exe',
 None,
 'https://github.com/llvm/llvm-project/releases/download/llvmorg-15.0.7/LLVM-15.0.7-win64.exe',
 'https://github.com/llvm/llvm-project/releases/download/llvmorg-15.0.6/LLVM-15.0.6-win64.exe',
 'https://github.com/llvm/llvm-project/releases/download/llvmorg-15.0.5/LLVM-15.0.5-win64.exe',
 'https://github.com/llvm/llvm-project/releases/download/llvmorg-15.0.4/LLVM-15.0.4-win64.exe',
 'https://github.com/llvm/llvm-project/releases/download/llvmorg-15.0.3/LLVM-15.0.3-win64.exe',
 'https://github.com/llvm/llvm-project/releases/download/llvmorg-15.0.2/LLVM-15.0.2-win64.exe',
 'https://github.com/llvm/llvm-project/releases/download/llvmorg-15.0.1/LLVM-15.0.1-win64.exe']